# Prediction of music listening (Part IV): Full prediction

In this notebook, we are applying different models of Machine Learning to the whole data available from the kkbox challenge to make predictions whether a user will relisten to a stong.

In the first section, we are using TensorFlow to run the model. Then, we use a boosting algorithm called XGBoost.

## Acquiring data

In [0]:
!pip install --user -q kaggle

In [0]:
from google.colab import files
files.upload()

MessageError: ignored

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c kkbox-music-recommendation-challenge

!7za x train.csv.7z
!7za x members.csv.7z
!7za x songs.csv.7z

cp: cannot stat 'kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         
ERROR: No more files
train.csv.7z



System ERROR:
Unknown error -2147024872

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_U

## Pre-processing
We have to create a test set and a training set. We also do some data visualization.

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

df=pd.read_csv('train.csv')
df['song_num_listeners'] = df.groupby('song_id')['song_id'].transform('count')
df['user_num_listenings'] = df.groupby('msno')['msno'].transform('count')
df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_num_listeners,user_num_listenings
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,215,5511
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,1,622
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,4,622
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,1,622
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,412,5511


In [0]:
df_users = pd.read_csv('members.csv')
df_users.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [0]:
df_songs = pd.read_csv('songs.csv')
df_songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


Usual preprocessing of the data with joins...

In [0]:
df = pd.merge(df, df_songs, left_on = 'song_id', right_on = 'song_id').drop(['song_id'], axis = 1)
df = pd.merge(df, df_users, left_on = 'msno', right_on = 'msno').drop(['msno'], axis = 1)
df.head()

,source_system_tab,source_screen_name,source_type,target,song_num_listeners,user_num_listenings,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,explore,Explore,online-playlist,1,215,5511,206471,359,Bastille,Dan Smith| Mark Crew,NaN,52.0,1,0,NaN,7,20120102,20171005
1,explore,Explore,online-playlist,1,412,5511,187802,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,NaN,52.0,1,0,NaN,7,20120102,20171005
2,explore,Explore,online-playlist,1,1108,5511,247803,1259,Desiigner,Sidney Selby| Adnan Khan,NaN,52.0,1,0,NaN,7,20120102,20171005
3,explore,Explore,online-playlist,1,287,5511,181115,1011,Thomas Rhett,Thomas Rhett| Rhett Akins| Ben Hayslip,NaN,52.0,1,0,NaN,7,20120102,20171005
4,explore,Explore,online-playlist,0,738,5511,257369,465,OneRepublic,Ryan Tedder,NaN,52.0,1,0,NaN,7,20120102,20171005


In [0]:
# Dates to category
df['registration_init_time'] = df['registration_init_time'].astype('category')
df['expiration_date'] = df['expiration_date'].astype('category')

# Object data to category
for col in df.select_dtypes(include = ['object']).columns:
    df[col] = df[col].astype('category')
    
# Encoding categorical features
for col in df.select_dtypes(include = ['category']).columns:
    df[col] = df[col].cat.codes

In [0]:
# visualize
df.head()

,source_system_tab,source_screen_name,source_type,target,song_num_listeners,user_num_listenings,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,1,7,6,1,215,5511,206471,285,3277,14581,-1,52.0,1,0,-1,7,2131,989
1,1,7,6,1,412,5511,187802,2,4472,8485,-1,52.0,1,0,-1,7,2131,989
2,1,7,6,1,1108,5511,247803,90,8146,55579,-1,52.0,1,0,-1,7,2131,989
3,1,7,6,1,287,5511,181115,2,30636,59127,-1,52.0,1,0,-1,7,2131,989
4,1,7,6,0,738,5511,257369,344,22377,52683,-1,52.0,1,0,-1,7,2131,989


In [0]:
# test and train datasets
df_train, df_test = train_test_split(df, test_size = 0.2)

Considering the results we obtained in the previous section, we want to use a boosting algorithm, such as **XGBoost**. 

The algorithm uses DMatrix for its calculations. This format is optimized for XGBoost.

In [0]:
import xgboost as xgb
from xgboost import XGBClassifier
import numpy as np

dtrain = xgb.DMatrix(df_train[df_train.columns[df_train.columns != 'target']], df_train.target)
dtest = xgb.DMatrix(df_test[df_test.columns[df_test.columns != 'target']], df_test.target)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [0]:
params = {'objective': 'binary:hinge',
          'colsample_bytree': 0.3,
          'learning_rate': 0.1,
          'max_depth': 8,
          'alpha': 1,
          'tree_method': 'gpu_hist',
          'subsample': 0.5,
          'verbosity':2}
num_round = 400
progress = dict()
evals = [(dtrain, 'train'), (dtest, 'test')]
boost_model = xgb.train(params, dtrain, num_boost_round = num_round, evals = evals, evals_result = progress)

[0]	train-error:0.496592	test-error:0.49604
[1]	train-error:0.496592	test-error:0.49604
[2]	train-error:0.496592	test-error:0.49604
[3]	train-error:0.496592	test-error:0.49604
[4]	train-error:0.496592	test-error:0.49604
[5]	train-error:0.496592	test-error:0.49604
[6]	train-error:0.496588	test-error:0.496036
[7]	train-error:0.496302	test-error:0.495758
[8]	train-error:0.492998	test-error:0.492426
[9]	train-error:0.481163	test-error:0.480825
[10]	train-error:0.46901	test-error:0.468657
[11]	train-error:0.469004	test-error:0.468652
[12]	train-error:0.461418	test-error:0.461026
[13]	train-error:0.460961	test-error:0.460647
[14]	train-error:0.45564	test-error:0.455254
[15]	train-error:0.450398	test-error:0.450057
[16]	train-error:0.445738	test-error:0.445528
[17]	train-error:0.439289	test-error:0.439244
[18]	train-error:0.433449	test-error:0.433405
[19]	train-error:0.431063	test-error:0.430995
[20]	train-error:0.428628	test-error:0.428488
[21]	train-error:0.426562	test-error:0.42641
[22]	tr

We can conclude that this method is not efficient because the test error is very high and doesn't decrease with a growing number of iterations.

Let's apply XGBoost to a Random Forest method.

In [0]:
params = {
  'colsample_bynode': 0.8,
  'learning_rate': 1,
  'max_depth': 5,
  'num_parallel_tree': 100,
  'objective': 'binary:logistic',
  'subsample': 0.8,
  'tree_method': 'gpu_hist',
  'verbosity': 2
}
num_round = 400
progress = dict()
evals = [(dtrain, 'train'), (dtest, 'test')]
rf_model = xgb.train(params, dtrain, num_boost_round = num_round, evals = evals, evals_result = progress)

[0]	train-error:0.351795	test-error:0.351293
[1]	train-error:0.350111	test-error:0.34974
[2]	train-error:0.34773	test-error:0.347365
[3]	train-error:0.347282	test-error:0.346925
[4]	train-error:0.346025	test-error:0.345613
[5]	train-error:0.345303	test-error:0.344999
[6]	train-error:0.344356	test-error:0.344063
[7]	train-error:0.343918	test-error:0.343667
[8]	train-error:0.34324	test-error:0.343038
[9]	train-error:0.342689	test-error:0.342546
[10]	train-error:0.342088	test-error:0.341928
[11]	train-error:0.341653	test-error:0.341544
[12]	train-error:0.341362	test-error:0.34124
[13]	train-error:0.340999	test-error:0.34086
[14]	train-error:0.34062	test-error:0.340595
[15]	train-error:0.340239	test-error:0.340166
[16]	train-error:0.339893	test-error:0.339781
[17]	train-error:0.339546	test-error:0.339432
[18]	train-error:0.339269	test-error:0.33911
[19]	train-error:0.338953	test-error:0.338747
[20]	train-error:0.338638	test-error:0.338437
[21]	train-error:0.338359	test-error:0.338134
[22]	

KeyboardInterrupt: ignored